In [18]:
import cv2
from openpyxl import load_workbook
import copy

In [5]:
op_xlsx_path = '/Users/ivory/Documents/1/openpose_data/excel/'
op_joint_path = '/Users/ivory/Documents/1/openpose_data/2d_joint/'
pp11_video_path = '/Users/ivory/Documents/1/render/pp11/'
task_list = [
    'gait1_front',
    'gait1_rear',
    'gait2_front',
    'gait2_rear',
    'obstacle_high_front',
    'obstacle_high_rear',
    'obstacle_low_front',
    'obstacle_low_rear',
    'tug_front',
    'tug_low',
    'walk_fast_front',
    'walk_fast_rear',
    'walk_preferred_front',
    'walk_preferred_rear',
    'walk_slow_front',
    'walk_slow_rear'
]

In [6]:
TASK_IDX = 0
ACTIVE_SHEET_NAME = 'OpIdx0'
joint_data_path = op_xlsx_path + task_list[TASK_IDX] + '.xlsx'
video_path = pp11_video_path + task_list[TASK_IDX] + '.mp4'

In [19]:
op_wb = load_workbook(joint_data_path)
xlsx_sheet_name_list = op_wb.sheetnames

In [20]:
xlsx_sheet_name_list

['OpIdx0', 'OpIdx1', 'OpIdx2']

In [28]:
ws = op_wb['OpIdx0']
row = ws[3]
row_values = [cell.value for cell in row]

In [29]:
row_values

[2,
 904.213,
 389.141,
 0.987566,
 880.508,
 421.419,
 0.923053,
 857.007,
 415.607,
 0.944219,
 836.489,
 462.539,
 0.931456,
 839.333,
 506.694,
 0.963739,
 903.966,
 427.306,
 0.890706,
 895.293,
 468.516,
 0.248776,
 901.139,
 495.007,
 0.22434,
 871.699,
 506.603,
 0.882347,
 857.006,
 503.837,
 0.945053,
 842.277,
 574.343,
 0.894563,
 833.285,
 633.297,
 0.859629,
 883.508,
 506.698,
 0.948753,
 865.89,
 574.479,
 0.891346,
 856.967,
 639.15,
 0.908586,
 903.904,
 386.16,
 0.963619,
 906.921,
 388.895,
 0.358017,
 883.374,
 388.891,
 0.974086,
 0,
 0,
 0,
 868.754,
 650.946,
 0.895143,
 877.477,
 647.983,
 0.90525,
 851.086,
 644.969,
 0.718175,
 839.447,
 648.056,
 0.770927,
 833.381,
 647.984,
 0.781551,
 827.541,
 642.159,
 0.819582]

In [33]:
a = [1,2,3,4,5]

In [34]:
q,w,e,r,t = a

In [41]:
# 예제 2차원 배열
data = [
    [5, 1, 2],
    [10, 3, 4],
    [15, 5, 6],
    [20, 7, 8]
]

target = 15
found_idx = -1
for idx, row in enumerate(data):
    if row[0] == target:
        found_idx = idx
        break

In [42]:
found_idx

2

In [52]:
reID_dict = {'0':[1,2,4], '1':[1,3,5,1]}

In [46]:
list(reID_dict.keys())

['0', '1']

In [47]:
a = [1,2,34,6,67,5]
max(a)

67

In [50]:
a.index(max(a))

4

In [53]:
for key in reID_dict:
    print(key)

0
1


In [54]:
from openpyxl import load_workbook, Workbook
import json

# Variables #
#Const
op_xlsx_path = '/Users/ivory/Documents/1/openpose_data/excel/'
roi_path = '/Users/ivory/Documents/1/pre_roi/'
pp11_video_path = '/Users/ivory/Documents/1/render/pp11/'
task_list = [
    'gait1_front',
    'gait1_rear',
    'gait2_front',
    'gait2_rear',
    'obstacle_high_front',
    'obstacle_high_rear',
    'obstacle_low_front',
    'obstacle_low_rear',
    'tug_front',
    'tug_low',
    'walk_fast_front',
    'walk_fast_rear',
    'walk_preferred_front',
    'walk_preferred_rear',
    'walk_slow_front',
    'walk_slow_rear'
]
xlsx_sheet_name_list = []
#controll variables
TASK_IDX = 0
ACTIVE_SHEET_NAME = 'OpIdx1'
joint_data_path = op_xlsx_path + task_list[TASK_IDX] + '.xlsx'
video_path = pp11_video_path + task_list[TASK_IDX] + '.mp4'
roi_json_path = roi_path + 'pp11/' +task_list[TASK_IDX] + '.json'
#####################

# fuction part #
## private function ##
def _is_point_in_roi(roi:tuple, point: tuple):
    """
    point가 ROI 내부에 존재하는지 판별
    Args:
        roi(tuple): (pt1_x, pt1_y, pt2_x, pt2_y)
        point(tuple): (x, y)
    
    Returns:
        int: point가 ROI내부에 존재하면 1, 그렇지 않으면 0.
    """
    pt1_x, pt1_y, pt2_x, pt2_y = roi
    x, y = point
    
    if pt1_x <= x <= pt2_x and pt1_y <= y <= pt2_y:
        return 1
    else:
        return 0
def _find_idx(data, target):
    """
    Args:
        data: 2차원 배열
        target: 타겟 숫자
    """
    for i, row in enumerate(data):
        if row[0] == target:
            return i
    return -1

## public finction ##
def re_identification2(roi_dict: dict, op_data_wb: Workbook, sheet_name_list: list[str]) -> dict:
    """
    ROI data를 이용해서 OpenPose의 people idx를 재분류하는 함수.

    Args:
        roi_dict (dict): 각 프레임에 대한 bounding box 정보.
        op_data_wb (Workbook): OpenPose 데이터를 포함한 엑셀 워크북.
        sheet_name_list (list[str]): 워크북의 시트 이름 목록.

    Returns:
        dict: ROI별로 재분류된 프레임 데이터.
    """
    reID_dict = {'0': [], '1': []}
    op_dict = {}

    # 각 시트에서 데이터 읽어오기
    for cur_sheet in sheet_name_list:
        ws = op_data_wb[cur_sheet]
        op_dict[cur_sheet] = [list(row) for row in ws.iter_rows(values_only=True)]

    # ROI별로 데이터 처리
    for roi_key in roi_dict:
        for cur_roi in roi_dict[roi_key]:
            cur_roi_frame_number, pt1_x, pt1_y, pt2_x, pt2_y = cur_roi

            op_candidate = []
            for sheet_name in sheet_name_list:
                op_idx = _find_idx(op_dict[sheet_name], cur_roi_frame_number)
                if op_idx != -1:  # 프레임 번호가 시트에 존재하는 경우에만 추가
                    op_candidate.append(op_dict[sheet_name][op_idx])

            if not op_candidate:  # 후보가 없으면 다음 ROI로 이동
                continue

            # joint 위치가 가장 많이 포함된 후보 찾기
            cnt_list = [0] * len(op_candidate)  # 후보 목록의 조인트 포함 개수 저장
            for idx, each_row in enumerate(op_candidate):
                for i in range(1, 75, 3):  # x, y 좌표만 검사
                    joint_x = each_row[i]
                    joint_y = each_row[i + 1]
                    if joint_x is not None and joint_y is not None:  # None 값 확인
                        cnt_list[idx] += _is_point_in_roi((pt1_x, pt1_y, pt2_x, pt2_y), (joint_x, joint_y))

            if cnt_list and max(cnt_list) > 0:  # 최대 포함 개수가 0보다 클 때만 추가
                max_idx = cnt_list.index(max(cnt_list))
                reID_dict[roi_key].append(op_candidate[max_idx])

    return reID_dict

def re_identification(roi_dict:dict, op_data_wb:Workbook, sheet_name_list:list[str]):
    """
    ROI data를 이용해서 OpenPose의 people idx를 재분류하는 함수.
    arg:
        roi_dict (dict): 각 프레임에 대한 bounding box. idx는 0또는 1
            format -> {
                "{0 || 1}":[
                    [0, x1, y1, x2, y2],
                    [1, x1, y1, x2, y2],
                    ...
                    [t-1, x1, y1, x2, y2]
                ]
            }
        op_data_wb (Workbook):
            data format -> 
                row-1: [0, 25 * 3], 
                row-2: [1, 25 * 3],
                row-3: [2, 25 * 3],
                ...
                row-t: [t-1, 25 * 3]
            
        sheet_name_list (list[str]): ["sheet1", "sheet2", ...]
        
        threshold (int): 분류 기준 임계치
    return:
        reID_dict (dict)
            format -> {
                "0": [
                    [0, 25 * 3],
                    [1, 25 * 3],
                    ..
                    [t-1, 25 * 3]
                ]
                "1": [
                    ...
                ]
            }
    """
    reID_dict = {'0':[], '1':[]}
    op_dict = {}
    '''
    op_dict = {
        sheet_name: [[frmae number, 25*3], [], []... []],
        sheet_name: [[frmae number, 25*3], [], []... []],
        sheet_name: [[frmae number, 25*3], [], []... []],
                }
    '''
    
    roi0_data_list = roi_dict['0']
    roi1_data_list = roi_dict['1']
    
    #op_data_wb를 dict 형태로 변환
    for cur_sheet in sheet_name_list:
        ws = op_data_wb[cur_sheet]
        op_dict[cur_sheet] = []
        
        for row in ws.iter_rows(values_only=True):
            op_dict[cur_sheet].append(list(row))
    ############################
    
    #roi idx 0번
    for cur_roi in roi0_data_list:
        cur_roi_frame_number, pt1_x, pt1_y, pt2_x, pt2_y = cur_roi
        
        op_candidate = []
        for sheet_name in sheet_name_list:
            op_idx = _find_idx(op_dict[sheet_name], cur_roi_frame_number)
            if op_idx != -1:
                op_candidate.append(op_dict[sheet_name][op_idx])
        
        #현재 op_candidate에 후보 목록들이 들어가 있는 상태 e.g. [[sheet1],[sheet2],[sheet3],[sheet4]]
        #지금부터 joint pose가 몇개 들어가 있는지 세야함.
        cnt_list = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0] #각 sheet의 row joint 들어가는 개수
        for idx, each_row in enumerate(op_candidate):
            for i in range(1, 75, 3):
                joint_x = each_row[i]
                joint_y = each_row[i+1]
                
                cnt_list[idx] +=_is_point_in_roi((pt1_x, pt1_y, pt2_x, pt2_y), (joint_x, joint_y))
        
        max_idx = cnt_list.index(max(cnt_list))
        reID_dict['0'].append(op_candidate[max_idx])
        
    #roi idx 1번
    for cur_roi in roi1_data_list:
        cur_roi_frame_number, pt1_x, pt1_y, pt2_x, pt2_y = cur_roi
    
        op_candidate = []
        for sheet_name in sheet_name_list:
            op_idx = _find_idx(op_dict[sheet_name], cur_roi_frame_number)
            if op_idx != -1:
                op_candidate.append(op_dict[sheet_name][op_idx])
        
        #현재 op_candidate에 후보 목록들이 들어가 있는 상태 e.g. [[sheet1],[sheet2],[sheet3],[sheet4]]
        #지금부터 joint pose가 몇개 들어가 있는지 세야함.
        cnt_list = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0] #각 sheet의 row joint 들어가는 개수
        for idx, each_row in enumerate(op_candidate):
            for i in range(1, 75, 3):
                joint_x = each_row[i]
                joint_y = each_row[i+1]
                
                cnt_list[idx] +=_is_point_in_roi((pt1_x, pt1_y, pt2_x, pt2_y), (joint_x, joint_y))
        
        max_idx = cnt_list.index(max(cnt_list))
        reID_dict['1'].append(op_candidate[max_idx])
            
    return reID_dict
    
#####################

# logic #
with open(roi_json_path, 'r') as file:
    roi_dict = json.load(file)

op_wb = load_workbook(joint_data_path)
xlsx_sheet_name_list = op_wb.sheetnames

reID_dict = re_identification2(roi_dict, op_wb, xlsx_sheet_name_list)

In [57]:
reID_dict.keys()

dict_keys(['0', '1'])

In [58]:
len(reID_dict['0'])

250

In [59]:
len(reID_dict['1'])

228

In [60]:
reID_dict['0']

[[0,
  1060.09,
  435.981,
  0.901856,
  1072.01,
  462.628,
  0.915146,
  1051.25,
  459.793,
  0.921842,
  1045.27,
  503.652,
  0.899884,
  1036.48,
  530.284,
  0.862952,
  1095.52,
  465.486,
  0.902061,
  1098.41,
  506.816,
  0.89208,
  1092.33,
  547.984,
  0.934578,
  1065.9,
  542.102,
  0.842505,
  1048.3,
  539.137,
  0.877267,
  1045.47,
  598.078,
  0.944398,
  1045.32,
  651.058,
  0.910336,
  1074.77,
  545.031,
  0.862361,
  1071.82,
  600.82,
  0.922122,
  1068.88,
  653.9,
  0.825812,
  1060.06,
  433.058,
  0.382101,
  1066.01,
  433.045,
  0.904618,
  0,
  0,
  0,
  1077.78,
  433.205,
  0.908217,
  1051.21,
  668.571,
  0.636262,
  1063.02,
  668.707,
  0.662145,
  1068.91,
  662.755,
  0.696948,
  1027.69,
  668.508,
  0.829021,
  1024.81,
  662.754,
  0.789562,
  1048.28,
  662.763,
  0.748019],
 [1,
  1060.09,
  433.272,
  0.894494,
  1072.01,
  462.577,
  0.923116,
  1051.25,
  459.727,
  0.932442,
  1045.26,
  500.854,
  0.891087,
  1036.51,
  530.218,
  0.87

# ALL-In-One

In [61]:
from dragon import dragonV
import json

In [ ]:
#Variables
##Const
openpose_path = '/Users/ivory/Documents/1/openpose_pp11/'

#json 통합 경로
op_joint_path = '/Users/ivory/Documents/1/openpose_data/2d_joint/'
op_excel_path = '/Users/ivory/Documents/1/openpose_data/excel/'
roi_path = '/Users/ivory/Documents/1/pre_roi/pp11/'
task_list = [
    'gait1_front',
    'gait1_rear',
    'gait2_front',
    'gait2_rear',
    'obstacle_high_front',
    'obstacle_high_rear',
    'obstacle_low_front',
    'obstacle_low_rear',
    'tug_front',
    'tug_low',
    'walk_fast_front',
    'walk_fast_rear',
    'walk_preferred_front',
    'walk_preferred_rear',
    'walk_slow_front',
    'walk_slow_rear'
]
##Control
TASK_IDX = 3

In [ ]:
#logic
## openpose jsons -> json
openpose_json_folder_path = openpose_path + task_list[TASK_IDX]
json_list = dragonV.get_jsons_list(openpose_json_folder_path)

op_dict = {}
for cur_frame_number, each_json in enumerate(json_list):
    with open(openpose_json_folder_path + '/' + each_json, 'r') as file:
        cur_json = json.load(file)
    
    #하나의 json file에 존재하는 사람 수
    people_list = cur_json['people']
    people_len = len(people_list)
    
    for people_idx, cur_people in enumerate(people_list):
        joint_2d = cur_people['pose_keypoints_2d']
        joint_2d.insert(0, cur_frame_number)
        
        if people_idx in op_dict:
            op_dict[people_idx].append(joint_2d)
        else:
            op_dict[people_idx] = []
            op_dict[people_idx].append(joint_2d)

with open(op_joint_path + '/' + task_list[TASK_IDX] + '.json', 'w', encoding='utf-8') as file:
    json.dump(op_dict, file, ensure_ascii=False, indent=4)
    
## json -> excel
op_json = op_joint_path + task_list[TASK_IDX] + '.json'
op_xlsx = op_xlsx_path + task_list[TASK_IDX] + '.xlsx'
#####################

with open(op_json, 'r') as file:
    op_json_dict = json.load(file)

wb = Workbook()
for people_idx in op_json_dict:
    print(people_idx)
    sheet = wb.create_sheet(title=f'OpIdx{people_idx}')
    wb.active = wb[f'OpIdx{people_idx}']
    ws = wb.active
    
    for row in op_json_dict[f'{people_idx}']:
        ws.append(row)

wb.save(op_xlsx)

#reidentification
